In [1]:
from A_Star import *
import copy


a, N, M, coord_robinet, coord_canal = read_file("date.in")
add_costs(a, N, M)
costs = np.array(a).flatten()
print_matrix_weights(a)

# print_matrix(a)
# print(a)
# print(coord_canal)

[ 0 ,   0 ,   0 ,   0 ,   3 ,   0 ,   0 ,   0 ,   1 ]
[ 2 ,   3 ,   0 ,   3 ,   4 ,   4 ,   3 ,   2 ,   0 ]
[ 0 ,   5 ,   4 ,   0 ,   0 ,   5 ,   0 ,   0 ,   0 ]
[ 2 ,   4 ,   0 ,   0 ,   3 ,   0 ,   3 ,   0 ,   1 ]
[ 0 ,   0 ,   5 ,   5 ,   4 ,   0 ,   4 ,   4 ,   0 ]
[ 0 ,   2 ,   4 ,   4 ,   0 ,   3 ,   3 ,   0 ,   0 ]


In [2]:
def coord_to_node(coord_x, coord_y, M):
    return M*coord_x + coord_y

def node_to_coord(node, M):
    return (int(node / M), node % M)

def adiacency_matrix(N, M):
    a_mat = np.zeros((N * M, N * M))
#     print(N,M)
    for i in range(N):
        for j in range(M):
            node_number = coord_to_node(i, j, M)
#             print(i, j, node_number)
            if j == 0:
                a_mat[node_number][node_number + 1] = 1
            elif j == M - 1:
                a_mat[node_number][node_number - 1] = 1
            else:
                a_mat[node_number][node_number + 1] = a_mat[node_number][node_number - 1] = 1
                
            if i == N - 1:
#                 print(node_number, node_number - M)
                a_mat[node_number][node_number - M] = 1
            elif int(node_number / M) == 0:
                a_mat[node_number][node_number + M] = 1                    
            else:
                a_mat[node_number][node_number + M] = a_mat[node_number][node_number - M] = 1
    return a_mat

def adiacency_matrix_weights(N, M, costs):
    a_mat = np.zeros((N * M, N * M))
#     print(N,M)
    for i in range(N):
        for j in range(M):
            node_number = coord_to_node(i, j, M)
#             print(i, j, node_number)
            if j == 0:
                a_mat[node_number][node_number + 1] = costs[node_number + 1] + 1
            elif j == M - 1:
                a_mat[node_number][node_number - 1] = costs[node_number - 1] + 1
            else:
                a_mat[node_number][node_number + 1] = costs[node_number + 1] + 1
                a_mat[node_number][node_number - 1] = costs[node_number - 1] + 1
                
            if i == N - 1:
#                 print(node_number, node_number - M)
                a_mat[node_number][node_number - M] = costs[node_number - M] + 1
            elif i == 0:
                a_mat[node_number][node_number + M] = costs[node_number + M] + 1            
            else:
                a_mat[node_number][node_number + M] = costs[node_number + M] + 1
                a_mat[node_number][node_number - M] = costs[node_number - M] + 1
    return a_mat

a_mat = adiacency_matrix(N, M)
a_mat_weighted = adiacency_matrix_weights(N, M, costs)

In [3]:
a_mat[4][3]

1.0

In [4]:
def build_vect_h(matrix, scopes = None, N = None, M = None):
    vect_h = []
    if N is None or M is None:
        N = len(matrix)
        M = len(matrix[0])
    for i in range(N):
        for j in range(M):
            cost = 0
            probably_closest_node = scopes[0]
            i_node = probably_closest_node[0]
            j_node = probably_closest_node[1]
            j_cost = abs(j_node - j)
            i_cost = abs(i_node - i)
#             print((i, i_node), (j, j_node))
            for line in range(min(i, i_node), max(i, i_node) + 1):
                for column in range(min(j, j_node), max(j, j_node) + 1):
#                     print(matrix[line][column])
                    if matrix[line][column] > 0:
                        cost += matrix[line][column]
            cost += i_cost + j_cost
            vect_h.append(cost)
    return vect_h

def print_vect_h(vect_h):
    print(np.array(vect_h).reshape(N,M))

In [5]:
vect_h = build_vect_h(a, coord_canal, N, M)
print_vect_h(vect_h)

[[66 61 46 32 19 32 46 53 56]
 [62 57 42 28 15 28 42 49 51]
 [49 46 34 20 10 19 30 35 37]
 [39 36 29 19  9 13 24 29 31]
 [29 28 25 15  5  9 17 22 23]
 [14 13 10  5  0  4  8  9 10]]


In [6]:
noduri = list(range(0, N * M))
# print(noduri)
start = coord_to_node(coord_robinet[0],coord_robinet[1],M)
scopuri = [coord_to_node(scope[0], scope[1], M) for scope in coord_canal]
# print(scopuri)

In [7]:
class NodParcurgere:
    graf = None  # static

    def __init__(self, id, info, parinte, cost, h):
        self.id = id  # este indicele din vectorul de noduri
        self.info = info
        self.parinte = parinte  # parintele din arborele de parcurgere
        self.g = cost  # costul de la radacina la nodul curent
        self.h = h
        self.f = self.g + self.h

    def obtineDrum(self):
        l = [self.info]
        nod = self
        while nod.parinte is not None:
            l.insert(0, nod.parinte.info)
            nod = nod.parinte
        return l

    def afisDrum(self):  # returneaza si lungimea drumului
        drum = self.obtineDrum()
        print(("->").join([str(node) for node in drum]))
        print("Cost: ", self.g)
        return len(drum)

    def contineInDrum(self, infoNodNou):
        nodDrum = self
        while nodDrum is not None:
            if (infoNodNou == nodDrum.info):
                return True
            nodDrum = nodDrum.parinte

        return False

    def __repr__(self):
        sir = ""
#         print(self.info)
        sir += str(self.info) + "("
        sir += "id = {}, ".format(self.id)
        sir += "drum="
        drum = self.obtineDrum()
        sir += ("->").join([str(node) for node in drum])
        sir += " g:{}".format(self.g)
        sir += " h:{}".format(self.h)

        sir += " f:{})".format(self.f)
        return sir


class Graph:  # graful problemei
    def __init__(self, noduri, matriceAdiacenta, costuri, start, scopuri, lista_h):
        self.noduri = noduri
        self.matriceAdiacenta = matriceAdiacenta
        self.costuri = costuri
        self.nrNoduri = len(matriceAdiacenta)
        self.start = start
        self.scopuri = scopuri
        self.lista_h = lista_h

    def indiceNod(self, n):
        return self.noduri.index(n)

    def testeaza_scop(self, nodCurent):
#         print(nodCurent.info in self.scopuri)
        return nodCurent.info in self.scopuri

    # va genera succesorii sub forma de noduri in arborele de parcurgere
    def genereazaSuccesori(self, nodCurent):
        listaSuccesori = []
        for i in range(self.nrNoduri):
            if self.matriceAdiacenta[nodCurent.id][i] == 1 and not nodCurent.contineInDrum(self.noduri[i]):
                nodNou = NodParcurgere(i, self.noduri[i], nodCurent, nodCurent.g + self.costuri[nodCurent.id],
                                       self.calculeaza_h(self.noduri[i]))
                listaSuccesori.append(nodNou)
        return listaSuccesori

    def calculeaza_h(self, infoNod):
        return self.lista_h[self.indiceNod(infoNod)]

    def __repr__(self):
        sir = ""
        for (k, v) in self.__dict__.items():
            sir += "{} = {}\n".format(k, v)
        return sir
    
def a_star(gr, nrSolutiiCautate = 100, showQueue = False):
    # in coada vom avea doar noduri de tip NodParcurgere (nodurile din arborele de parcurgere)
    copie_nrSolutiiCautate = nrSolutiiCautate
    c = [NodParcurgere(gr.indiceNod(gr.start), gr.start, None, 0, gr.calculeaza_h(gr.start))]
    best_road = c[0]
    while len(c) > 0:
        if showQueue:
            print("Coada actuala: " + str(c))
#             input()
        nodCurent = c.pop(0)
#         print("test")
        if gr.testeaza_scop(nodCurent):
#             print("Solutie: ")
#             nodCurent.afisDrum()
            if nodCurent.g < best_road.g or best_road.g == 0:
                best_road = nodCurent
#             print("\n----------------\n")
#             input()
            nrSolutiiCautate -= 1
            if nrSolutiiCautate == 0:
                print(f"Cea mai buna solutie din {copie_nrSolutiiCautate} de solutii: ")
                best_road.afisDrum()
                return best_road
            
        lSuccesori = gr.genereazaSuccesori(nodCurent)
        for s in lSuccesori:
            i = 0
            gasit_loc = False
            for i in range(len(c)):
                # diferenta fata de UCS e ca ordonez dupa f
                if c[i].f > s.f:
                    gasit_loc = True
                    break
                elif c[i].f == s.f:
                    if s.g <= c[i].g:
                        s, c[i] = c[i], s
                    gasit_loc = True
                    break
            if gasit_loc:
                c.insert(i, s)
            else:
                c.append(s)
    
    print(f"Cea mai buna solutie dupa ~{copie_nrSolutiiCautate - nrSolutiiCautate} solutii: ")
    best_road.afisDrum()
    return best_road

In [8]:
gr = Graph(noduri, a_mat, costs, start, scopuri, vect_h)
NodParcurgere.graf = gr
best_road = a_star(gr, showQueue = False)
print("done")

Cea mai buna solutie din 100 de solutii: 
0->1->2->3->12->21->22->31->32->41->50->49
Cost:  9
done


In [9]:
destroyed_walls = []
for elem in best_road.obtineDrum():
    if costs[elem] > 0:
        destroyed_walls.append(elem)
        
print(destroyed_walls)

[12, 31, 50]


In [10]:
def BFS(a, start, M, N):
    nodes = []
    # Visited vector to so that a
    # vertex is not visited more than
    # once Initializing the vector to
    # false as no vertex is visited at
    # the beginning
    vertixes = M * N
    visited = [False] * vertixes
    q = [start]

    # Set source as visited
    visited[start] = True

    while q:
        vis = q[0]

        # Print current node
#         print(vis, end = ' ')
        nodes.append(vis)
        q.pop(0)

        # For every adjacent vertex to
        # the current vertex
        for i in range(vertixes):
            if ((a[vis][i] == 1 or a[vis][i] == -1) and (not visited[i])):

                # Push the adjacent node
                # in the queue
                q.append(i)

                # set
                visited[i] = True
    return nodes

In [11]:
flooded_nodes = BFS(a_mat_weighted, start, M, N)
print(flooded_nodes)
print_matrix(a)

[0, 1, 2, 3, 11]
oooo#ooo#
##o#####o
o##oo#ooo
##oo#o#o#
oo###o##o
o###o##oo


In [12]:
flooded_coords = [node_to_coord(node, M) for node in flooded_nodes]
print(flooded_coords)

[(0, 0), (0, 1), (0, 2), (0, 3), (1, 2)]


In [13]:
def flood(flooded_coords, a_mat):
    for (x,y) in flooded_coords:
        a_mat[x][y] = -1
    print_matrix(a_mat)
    
a_copy = copy.deepcopy(a)
print_matrix(a)
print("\n")


flood(flooded_coords, a_copy)

oooo#ooo#
##o#####o
o##oo#ooo
##oo#o#o#
oo###o##o
o###o##oo


iiii#ooo#
##i#####o
o##oo#ooo
##oo#o#o#
oo###o##o
o###o##oo


In [14]:
# costs[12] = 0
# a_copy_mat_weighted = adiacency_matrix_weights(N, M, costs)
# flooded_nodes = BFS(a_copy_mat_weighted, start, M, N)
# print(flooded_nodes)
# print_matrix(a_copy)
# # flooded_coords = [node_to_coord(node, M) for node in flooded_nodes]
# print("\n")
# flood(flooded_coords, a_copy)

In [15]:
def next_state(start, M, N, last_a_state, costs):
    last_state_mat_weighted = adiacency_matrix_weights(N, M, costs)
    flooded_nodes = BFS(last_state_mat_weighted, start, M, N)
    last_a_state_copy = copy.deepcopy(last_a_state)
#     print(flooded_nodes)
    flooded_coords = [node_to_coord(node, M) for node in flooded_nodes]
    flood(flooded_coords, last_a_state_copy)
    return last_a_state_copy

In [16]:
# destroyed_coords = [node_to_coord(node, M) for node in destroyed_walls]
# print(destroyed_coords)
# print_matrix(a)
print("1)")
state = copy.deepcopy(a)
flood(flooded_coords, state)
# print(costs)
new_costs = copy.deepcopy(costs)
for i in range(len(destroyed_walls)):
    cost = destroyed_walls[i]
    print("Eliminăm obstacolul de la {}.".format(node_to_coord(cost, M)))
    print(f"\n{i + 2})")
    new_costs = [new_costs[i] if i != cost else 0 for i in range(len(new_costs))]
#     print(new_costs)
    state = next_state(start, M, N, state, new_costs)
    


1)
iiii#ooo#
##i#####o
o##oo#ooo
##oo#o#o#
oo###o##o
o###o##oo
Eliminăm obstacolul de la (1, 3).

2)
iiii#ooo#
##ii####o
o##ii#ooo
##ii#o#o#
oo###o##o
o###o##oo
Eliminăm obstacolul de la (3, 4).

3)
iiii#ooo#
##ii####o
o##ii#ooo
##iiii#o#
oo###i##o
o###o##oo
Eliminăm obstacolul de la (5, 5).

4)
iiii#ooo#
##ii####o
o##ii#ooo
##iiii#o#
oo###i##o
o###ii#oo
